In [1]:
import requests
from PIL import Image
from io import BytesIO
import tensorflow as tf
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from tensorflow.keras import layers, models
from google.colab import files

In [14]:
uploaded = files.upload()

Saving df_after_text.csv to df_after_text.csv


In [2]:
df = pd.read_csv('df_after_text.csv')

In [3]:
def get_tensor(url):
  response = requests.get(url)
  image = Image.open(BytesIO(response.content)).convert("RGB")
  float_caster = tf.cast(image, tf.float32)
  dims_expander = tf.expand_dims(float_caster, 0)
  resized = tf.image.resize(dims_expander, [224, 224])
  normalized = resized / 255.0

  return normalized

In [8]:
images = []
ids_not_parsed = []
for i in tqdm(range(df.shape[0])):
  link = df['img'][i]
  try:
    images.append(get_tensor(link))
  except:
    ids_not_parsed.append(df['id'][i])



  0%|          | 0/5148 [00:00<?, ?it/s]

In [16]:
df = df[~df['id'].isin(ids_not_parsed)]
images_tensor = tf.concat(images, axis = 0)

In [20]:
images_tensor.shape

TensorShape([5136, 224, 224, 3])

In [33]:
df_photos = pd.read_excel('df_after_text-2 (1.0) photo test.xlsx')
df_photos = df_photos[~df_photos['id'].isin(ids_not_parsed)].reset_index()
labels = list(df_photos[~df_photos['rating'].isna()]['rating'])
labeled_indices = list(df_photos[~df_photos['rating'].isna()].index)

In [11]:
X_train = images_array[labeled_indices]
y_train = np.array(labels)
unlabeled_indices = np.setdiff1d(np.arange(len(images_array)), labeled_indices)
X_unlabeled = images_array[unlabeled_indices]

In [37]:
labeled_indices = np.array(labeled_indices)
labels = np.array(labels).astype(int)
labeled_images = tf.gather(images_tensor, labeled_indices)

In [39]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # бинарная классификация
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(labeled_images, labels, epochs=10, batch_size=32)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 35s 3s/step - accuracy: 0.4658 - loss: 8.7882
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.5584 - loss: 0.6874
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.7000 - loss: 0.6156
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.6542 - loss: 0.5939
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.8897 - loss: 0.4976
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.8694 - loss: 0.4055
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.9300 - loss: 0.2806
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.9523 - loss: 0.1628
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.9974 - loss: 0.0826
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.9959 - loss: 0.0321


In [40]:
all_indices = np.arange(len(images_tensor))
unlabeled_indices = np.setdiff1d(all_indices, labeled_indices)
unlabeled_images = tf.gather(images_tensor, unlabeled_indices)

In [41]:
predicted_probs = model.predict(unlabeled_images)
predicted_labels = (predicted_probs > 0.5).astype(int).flatten()

149/149 ━━━━━━━━━━━━━━━━━━━━ 77s 507ms/step


In [62]:
df_photos.loc[df_photos['idx'].isin(unlabeled_indices), 'rating'] = predicted_labels

In [63]:
df_photos

,num,id,name,type,description,is_verified,color,brand,has_video,photo_count,...,item_rating,num_feedbacks,quantity,price,img,text_length,cluster,sentiment,rating,idx
0,1,251773651,Грызунок пульт для зубов,Прорезыватели,СУПЕР НОВИНКА!!! Теперь можно больше не прятат...,True,черный,KIDSLAB,True,16,...,4.9,49270,2469,25500,https://basket-16.wbbasket.ru/vol2517/part2517...,1119,1,0.074952,0.0,0
1,2,90705358,Мягкая игрушка Сяо геншин импакт,Мягкие игрушки,Мягкая игрушка Геншин Импакт! Игрушки выполнен...,False,синий,Плюшевый друг,False,1,...,4.8,2918,9,46800,https://basket-05.wbbasket.ru/vol907/part90705...,511,2,0.017000,0.0,1
2,3,73090209,Мягкая игрушка Ворон Бравл Старз,Мягкие игрушки,Мягкие игрушки выполнены в виде персонажей из ...,False,черный,LoveBeary,False,1,...,4.7,2548,18,46800,https://basket-05.wbbasket.ru/vol730/part73090...,934,1,0.028663,0.0,2
3,4,43079352,"Червячки деревянные, сортер для малышей магнит...",Сортеры,Дополнительный набор из 10 червячков для магни...,True,"зеленый, красный, голубой, оранжевый, желтый",Danila.toys,True,6,...,4.9,4014,2162,17100,https://basket-03.wbbasket.ru/vol430/part43079...,816,1,0.065792,0.0,3
4,5,217742416,Силиконовый прорезыватель грызунок пульт для з...,Прорезыватели,Силиконовый прорезыватель-пульт от для малышей...,False,глубокий черный,LIN FACTORY,False,13,...,4.9,2267,586,26100,https://basket-14.wbbasket.ru/vol2177/part2177...,2000,1,0.034376,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5131,5144,72078711,"Игровой набор ""Лето на даче""",Головоломки,"Игровой набор представлен в 6-ти вариантах: ""П...",True,NaN,Крона,True,9,...,5.0,821,709,61800,https://basket-05.wbbasket.ru/vol720/part72078...,1025,1,0.019000,0.0,5131
5132,5145,164443245,Among Us мини фигурки 1 шт во флоупаке,Фигурки-игрушки,Мини-фигурки Амонг Ас - идеальный подарок для ...,True,"красный, желтый, синий",Among us,True,8,...,4.7,956,845,28500,https://basket-11.wbbasket.ru/vol1644/part1644...,1810,1,0.036136,0.0,5132
5133,5146,138787230,"Картонный домик раскраска, большой развивающий...",Домики для детей,Картонный игровой развивающий Домик-раскраска ...,True,NaN,Brauberg,True,21,...,4.8,4046,439,157700,https://basket-10.wbbasket.ru/vol1387/part1387...,1588,1,0.044276,1.0,5133
5134,5147,165234550,Развивающие игрушки книжка шуршалка для новоро...,Книжки-игрушки,Большая развивающая книжка шуршалка с хвостика...,True,"фиолетовый меланж, голубой меланж, пурпурно-кр...",Zeimas,False,19,...,4.9,1800,491,66600,https://basket-11.wbbasket.ru/vol1652/part1652...,1180,1,0.017517,0.0,5134


In [67]:
df_photos['img'][4]

'https://basket-14.wbbasket.ru/vol2177/part217742/217742416/images/big/1.webp'

In [ ]:
from google.colab import files
files.download('имя_файла.csv')

In [70]:
df_photos.to_csv('final_df.csv', index = False)

In [72]:
np.save('images_tensor.npy', images_tensor.numpy())